In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


/scratch/user/siweicui/.conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from neural_compressor.config import PostTrainingQuantConfig
from neural_compressor.quantization import fit

In [3]:
# https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
float_model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", cache_dir="/scratch/user/siweicui/LLM/huggingface")


Loading checkpoint shards:   0%|                                            | 0/19 [00:00<?, ?it/s]

Loading checkpoint shards:   5%|█▉                                  | 1/19 [00:07<02:21,  7.86s/it]

Loading checkpoint shards:  11%|███▊                                | 2/19 [00:15<02:13,  7.85s/it]

Loading checkpoint shards:  16%|█████▋                              | 3/19 [00:22<01:58,  7.42s/it]

Loading checkpoint shards:  21%|███████▌                            | 4/19 [00:28<01:44,  6.95s/it]

...

Loading checkpoint shards:  84%|█████████████████████████████▍     | 16/19 [02:07<00:28,  9.58s/it]

Loading checkpoint shards:  89%|███████████████████████████████▎   | 17/19 [02:14<00:17,  8.75s/it]

Loading checkpoint shards:  95%|█████████████████████████████████▏ | 18/19 [02:26<00:09,  9.70s/it]

Loading checkpoint shards: 100%|███████████████████████████████████| 19/19 [02:36<00:00,  9.87s/it]

Loading checkpoint shards: 100%|███████████████████████████████████| 19/19 [02:36<00:00,  8.24s/it]

In [4]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

outputs = float_model.generate(inputs, max_new_tokens=5)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST] Yes, I do have


In [5]:
float_model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
        

In [6]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")


In [7]:
woq_conf = PostTrainingQuantConfig(approach="weight_only")


In [8]:
quantized_model = fit(model=float_model, conf=woq_conf)


2024-03-18 15:25:18 [INFO] Start auto tuning.


2024-03-18 15:25:18 [INFO] Quantize model without tuning!


2024-03-18 15:25:18 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.


2024-03-18 15:25:18 [INFO] Adaptor has 5 recipes.


2024-03-18 15:25:18 [INFO] 0 recipes specified by user.


...

2024-03-18 15:42:03 [INFO] Pass quantize model elapsed time: 1004043.04 ms


2024-03-18 15:42:03 [INFO] Save tuning history to /scratch/user/siweicui/mix_precision/nc_workspace/2024-03-18_15-19-30/./history.snapshot.


2024-03-18 15:42:03 [INFO] [Strategy] Found the model meets accuracy requirements, ending the tuning process.


2024-03-18 15:42:03 [INFO] Specified timeout or max trials is reached! Found a quantized model which meet accuracy goal. Exit.


2024-03-18 15:42:03 [INFO] Save deploy yaml to /scratch/user/siweicui/mix_precision/nc_workspace/2024-03-18_15-19-30/deploy.yaml


In [9]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")#.to("cuda")


In [10]:
quantized_model

PyTorchFXModel(
  (_model): MixtralForCausalLM(
    (model): MixtralModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x MixtralDecoderLayer(
          (self_attn): MixtralAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): MixtralRotaryEmbedding()
          )
          (block_sparse_moe): MixtralSparseMoeBlock(
            (gate): Linear(in_features=4096, out_features=8, bias=False)
            (experts): ModuleList(
              (0-7): 8 x MixtralBLockSparseTop2MLP(
                (w1): Linear(in_features=4096, out_features=14336, bias=False)
                (w2): Linear(in_features=14336, out_features=4096, bias=False)
                (w3): L

In [11]:
# https://github.com/intel/neural-compressor/issues/1208
quantized_model.model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
        

In [12]:
quantized_model.fp32_model.eval()
# 

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBLockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
        

In [13]:
# quantized_model.model.generate()

In [14]:
# quantized_model

# outputs = quantized_model.model(inputs)


In [15]:
quantized_model.save("/scratch/user/siweicui/mix_precision/quantized_model")

2024-03-18 15:46:27 [INFO] Save config file and weights of quantized model to /scratch/user/siweicui/mix_precision/quantized_model.


In [16]:
# https://github.com/huggingface/transformers/blob/main/src/transformers/modeling_outputs.py
outputs = quantized_model.model.generate(inputs, max_new_tokens=5)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST] I don't eat


In [17]:
# type(outputs)

In [18]:
# quantized_model
# q_model = AutoModelForCausalLM.from_pretrained(quantized_model)
# outputs2 = quantized_model.model(inputs)

In [19]:
# print(tokenizer.decode(outputs.last_hidden_state[0], skip_special_tokens=True))